In [1]:
import firebase_admin as admin
from firebase_admin import credentials, storage
import pandas as pd
import datetime
from firebase_admin import firestore

In [2]:
cred = credentials.Certificate("homebase-staging-832d6-firebase-adminsdk-5tm2f-9383944e6c.json")
#cred = credentials.Certificate("credentials.json")
app = admin.initialize_app(cred, {'storageBucket': "homebase-48c20.appspot.com"})

In [3]:
fs = firestore.client()
listing_documents = fs.collection('listings').get()
user_documents = fs.collection('users').get()

In [4]:
# Read excel file with desired deletions.
stale_deletes = pd.read_excel("listings_to_delete.xlsx", header=None)

                                                    0          1  \
0                         1119-Garnett-Pl-2-9A6K-WTNY 2021-03-31   
1               1500-Chicago-Avenue-Apt-609-YDYnZtj7S 2021-06-30   
2                       1500-Chicago-Avenue-jWRz4Ptci 2021-06-30   
3                       1500-Chicago-Avenue-v6UG5K9ek 2021-06-30   
4                      1521-W-Jonquil-Ter-3-pnp7ZIYY8 2021-06-30   
5             1637-Orrington-Avenue-Apt-801-uI9WdMdiP 2021-03-31   
6             1735-Chicago-Avenue-Apt.-810n-HAtNjgzM1 2021-04-01   
7                       1945-Sherman-Avenue-4WTucOwjV 2021-06-30   
8                    2121-Ridge-Ave-Unit-2j-p2HI9BXlG 2021-06-30   
9                    2125-Ridge-Ave-Unit-3c-6yn0tSa68 2021-06-30   
10                 2131.5-Ridge-Ave-Unit-1f-ZMmdspylb 2021-06-30   
11                      2525-Eastwood-Ave.--5WLTV0HKJ 2021-04-30   
12  716---720-Clark-Apartments,-716-Clark-Street-U... 2021-03-21   
13                     800-Custer-Avenue-2h-G9jf

In [4]:
# Grab the Listing URIs from the first column, convert to list.
#uris_delete = stale_deletes[0].tolist()
for user_doc in user_documents:
    fs.collection('users').document(user_doc.id).set({'listings' : []}, merge=True)

In [6]:
# For each listing URI, delete and clean up.
for uri in uris_delete:
    # Grab the UID before deleting.
    listing_uid = fs.collection('listings').document(uri).get().get('user')['uid']
    # Remove the listing from the user's list.
    curr_listings = fs.collection('users').document(listing_uid).get().get('listings')
    curr_listings.remove(uri)
    fs.collection('users').document(listing_uid).set({'listings': curr_listings}, merge=True)
    # Delete the listing.
    fs.collection('listings').document(uri).delete()
    

In [7]:
# Delete all listing photos from non-existing listings.
bucket = storage.bucket()
blobs = bucket.list_blobs(prefix="listing-images/")
for blob in blobs:
    for uri in uris_delete:
        if uri in blob.name:
            blob.delete()